In [76]:
from typing import List, Optional, NamedTuple
from queue import LifoQueue
import numpy as np
from functools import cached_property
import torch
from torch.distributions import Categorical
import timeit

In [77]:
class Resource:
    def __init__(self, capacity: int) -> None:
        self.capacity = capacity
        self.available = capacity
        self.last_event_time = 0
        self.queue = []
    
    def available_timestamp(self, amount: int) -> int:
        """Returns the earlist timestamp at which the requested amount is available."""
        assert amount <= self.capacity
        if amount == 0:
            return 0
        amount -= self.available
        if amount<=0:
            return self.last_event_time
        for release_time, release_amount in self.queue:
            amount -= release_amount
            if amount <= 0:
                return release_time
        raise Exception()
    
    def request(self, timestamp, amount, duration):
        assert timestamp >= self.last_event_time
        self.last_event_time = timestamp
        newqueue = []
        for release in self.queue:
            if release[0] <= timestamp:
                self.available += release[1]
            else:
                newqueue.append(release)
        newqueue.append((timestamp + duration, amount))
        self.queue = sorted(newqueue)
        self.available -= amount
        assert self.available >= 0, "Unable to fulfill this request"

class Activity:
    def __init__(self, index: int, duration: int = 0, resources: Optional[List[int]] = None) -> None:
        self.index = index
        self.duration = duration
        self.pred = []
        self.succ = []
        self.resources = resources or []
        self.latest_finish = 0xfffffff
        self.earlist_start = 0

    def add_successor(self, other):
        self.succ.append(other)
        other.pred.append(self)
    
    @cached_property
    def latest_start(self):
        return self.latest_finish - self.duration
    
    @cached_property
    def succ_closure(self) -> set[int]:
        closure = set()
        for act in self.succ:
            closure.add(act.index)
            closure.update(act.succ_closure)
        return closure

    @cached_property
    def pred_closure(self) -> set[int]:
        closure = set()
        for act in self.pred:
            closure.add(act.index)
            closure.update(act.pred_closure)
        return closure
    
    @cached_property
    def indegree(self):
        return len(self.pred)
    
    @cached_property
    def outdegree(self):
        return len(self.succ)

class RCPSPInstance:
    def __init__(self, activities: List[Activity], resource_capacity: List[int], max_total_time: Optional[int] = None):
        self.activities = activities
        self.capacity = resource_capacity
        self._calc_earlist_start_time()
        self._calc_latest_finish_time(max_total_time)
    
    @property
    def n(self):
        return len(self.activities)

    def __len__(self):
        return len(self.activities)
    
    def _calc_latest_finish_time(self, max_total_time: Optional[int] = None):
        if max_total_time is None:
            max_total_time = sum((act.duration for act in self.activities))
        self.activities[-1].latest_finish = max_total_time
        stack = LifoQueue()
        stack.put(self.activities[-1])
        while not stack.empty():
            node = stack.get()
            lf = node.latest_finish - node.duration
            for n in node.pred:
                if n.latest_finish > lf:
                    n.latest_finish = lf
                stack.put(n)
    
    def _calc_earlist_start_time(self):
        stack = LifoQueue()
        stack.put(self.activities[0])
        while not stack.empty():
            node = stack.get()
            es = node.earlist_start + node.duration
            for n in node.succ:
                if n.earlist_start < es:
                    n.earlist_start = es
                stack.put(n)
    
    @property
    def indegrees(self):
        return [act.indegree for act in self.activities]
    
    @property
    def outdegrees(self):
        return [act.outdegree for act in self.activities]

    @cached_property
    def adjlist(self):
        adjlist = []
        for act in self.activities:
            adjlist.append([i.index for i in act.succ])
        return adjlist
    
    def get_duration(self):
        return [i.duration for i in self.activities]

In [78]:
def readints(f) -> List[int]:
    return list(map(int, f.readline().strip().split()))

def read_RCPfile(filepath):
    with open(filepath) as f:
        n_jobs, n_resources = readints(f)
        resource_capacity = readints(f)
#         print('resource_capacity', resource_capacity)
        
        assert len(resource_capacity) == n_resources
        nodes = [Activity(i) for i in range(n_jobs)]
        for act in nodes:
            line = iter(readints(f))
            act.duration = next(line)
            act.resources = [next(line) for _ in range(n_resources)]
            n_successors = next(line)
            successors = list(line)  
#             print('act ind dur res', act.index, act.duration, act.resources)
            assert len(successors) == n_successors
            for succ_index in successors: 
                successor = nodes[succ_index - 1]
                act.add_successor(successor)
        assert f.read().strip() == "", 
    assert len(nodes[0].pred) == 0, 
    assert len(nodes[-1].succ) == 0, 

    return RCPSPInstance(nodes, resource_capacity)

SyntaxError: invalid syntax (3582259849.py, line 23)

In [79]:
def SSGS(rcpsp: RCPSPInstance, sequence: list[int]) -> list[int]:
    n = rcpsp.n
    valid = [True for _ in range(n)]
    indegrees = np.array(rcpsp.indegrees, dtype=np.int8)
    adjlist = [np.array(arr, dtype=np.uint16) for arr in rcpsp.adjlist]
    start_time = [0 for _ in range(n)]
    end_time = [0 for _ in range(n)]
    resources = [Resource(i) for i in rcpsp.capacity]
    for g in range(n):
        # fetch an activity to arrange time
        for j in sequence:
            if valid[j] and indegrees[j]<=0:
                break
        else:
            raise Exception("The precendence graph may contain a loop.")
        node = rcpsp.activities[j]
        requirement = node.resources
        
        # get earlist feasible start time
        earlist_start = max((end_time[p.index] for p in node.pred), default = node.earlist_start)
        arrange = max((r.available_timestamp(v) for r, v in zip(resources, requirement) if v>0), default=0)
        arrange = min(max(arrange, earlist_start), node.latest_start)

        # update states
        for r, v in zip(resources, requirement):
            if v>0:
                r.request(arrange, v, node.duration)
        start_time[j] = arrange
        end_time[j] = arrange + node.duration
        valid[j] = False
        indegrees[adjlist[j]] -= 1
#     print('ssgs start_time', start_time, sequence)
    return start_time

def SSGS_ordered(rcpsp: RCPSPInstance, sequence: list[int]) -> list[int]:
    n = rcpsp.n
    start_time = [0 for _ in range(n)]
    end_time = [0 for _ in range(n)]
    resources = [Resource(i) for i in rcpsp.capacity]
    for j in sequence:
        node = rcpsp.activities[j]
        requirement = node.resources

        # get earlist feasible start time
        earlist_start = max((end_time[p.index] for p in node.pred), default = node.earlist_start)
        arrange = max((r.available_timestamp(v) for r, v in zip(resources, requirement) if v>0), default=0)
        arrange = min(max(arrange, earlist_start), node.latest_start)

        # update states
        for r, v in zip(resources, requirement):
            if v>0:
                r.request(arrange, v, node.duration)
        start_time[j] = arrange
        end_time[j] = arrange + node.duration
    return start_time

class Solution(NamedTuple):
    route: np.ndarray
    schedule: np.ndarray
    cost: int

@torch.no_grad()
def nGRPWA_heuristic(rcpsp: RCPSPInstance):
    n = rcpsp.n
    column = torch.tensor([len(act.succ_closure) for act in rcpsp.activities])
    column = column - column.min() + 1
    return column.expand(n, n)

@torch.no_grad()
def nWRUP_heuristic(rcpsp: RCPSPInstance, omega = 0.5):
    n = rcpsp.n
    column = []
    for act in rcpsp.activities:
        value = omega * act.outdegree
        value += (1-omega) * sum(req/cap for req,cap in zip(act.resources, rcpsp.capacity))
        column.append(value)
    column = torch.tensor(column)
    column = column - column.min() + 1
    return column.expand(n, n)

class ACO_RCPSP:

    @torch.no_grad()
    def __init__(self, 
                 rcpsp: RCPSPInstance,
                 n_ants = 5, 
                 decay = 0.975,
                 alpha = 1.0,
                 beta = 2.0,
                 gamma = 0.0,
                 c = 0.6,
                 Q = 1.0,
                 min = 0.1,
                 elitist=False,
                 min_max=False,
                 pheromone: Optional[torch.Tensor] = None,
                 heuristic: Optional[torch.Tensor] = None,
                 device='cpu',
                 ):
        
        self.rcpsp = rcpsp
        self.n = rcpsp.n
        self.device = device
        self.adjlist = [np.array(i) for i in rcpsp.adjlist]

        self.n_ants = n_ants
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.Q = Q
        self.c = c
        self.elitist = elitist
        self.min_max = min_max
        self.min = min
        self.max = np.Infinity
        self.gamma = torch.tensor(gamma).to(device)

        self.epoch = 1

        if pheromone is not None:
            assert pheromone.shape == (rcpsp.n, rcpsp.n)
            self.pheromone = pheromone
        else:
            self.pheromone = torch.ones(rcpsp.n, rcpsp.n, dtype=torch.float32, device=device)
            if self.min_max:
                self.pheromone *= self.min
        
        if heuristic is not None:
            assert heuristic.shape == (rcpsp.n, rcpsp.n)
            self.heuristic = heuristic
        else:
            heuristic = nWRUP_heuristic(self.rcpsp, omega = 0.3)
            heuristic = heuristic / heuristic.max() * nGRPWA_heuristic(self.rcpsp)
            self.heuristic = heuristic.to(device)
        
        self.routes = torch.zeros(self.n_ants, self.n, dtype=torch.long, device=device)
        self.costs = torch.zeros(self.n_ants, dtype = torch.long, device = device)
        self.range_pop = torch.arange(self.n_ants, device=self.device)

        self.best_solution = Solution(np.array([]), np.array([]), 0xffffffff)

    @torch.no_grad()
    def run(self, n_iterations):
        for _ in range(n_iterations):
            self.construct_solutions()
            self.update_cost()
            self.update_pheromone()
            self.epoch += 1
            print(self.epoch, self.best_solution.cost)

        return self.best_solution
    
    def construct_solutions(self):
        probmat = self.pheromone.pow(self.alpha) * self.heuristic.pow(self.beta)
        not_visited = torch.ones(self.n_ants, self.n, dtype=torch.bool, device=self.device)
        indegrees = torch.tensor(self.rcpsp.indegrees, dtype=torch.int16).unsqueeze(0).expand(self.n_ants, self.n).to(self.device).contiguous()
        self.routes[:, 0] = prev = torch.tensor([0]).expand(self.n_ants)
        log_probs = []
        for k in range(self.n-1):
            # update status
            not_visited[self.range_pop, prev] = False
            for i, p in enumerate(prev):
                indegrees[i, self.adjlist[p]] -= 1
            # sample in topological order
            mask = not_visited * (indegrees == 0)

            if self.gamma < 0.05 or self.c == 1:
                # direct evaluation
                prob = probmat[prev] * mask
            else:
                # summation evaluation
                pheromone = self.pheromone[self.routes[:, :k+1]].reshape(self.n_ants, k+1, -1)
                if self.gamma != 1:
                    gamma = self.gamma.pow(torch.arange(k, -1, -1, device=self.device)).view(1,k+1,1)
                    pheromone = pheromone * gamma
                pheromone = pheromone.sum(dim=1) * mask
                summation_prob = pheromone.pow(self.alpha) * self.heuristic[prev].pow(self.beta)
                if self.c == 0:
                    prob = summation_prob
                else:
                    # balanced
                    direct_prob = probmat[prev] * mask
                    prob = self.c * direct_prob + (1-self.c) * summation_prob
            dist = Categorical(prob)
            self.routes[:, k+1] = prev = dist.sample()

    
    @torch.no_grad()
    def update_cost(self):
        schedules = []
        for i, route in enumerate(self.routes):
            schedule = SSGS_ordered(self.rcpsp, route.cpu().numpy())
            schedules.append(schedule)
            self.costs[i] = schedule[-1]
#             print(f"Cost for route {i}: {self.costs[i]}")
        bestindex = self.costs.argmin()
#         print(f"Best route index before update: {bestindex}, Cost: {self.costs[bestindex]}")
        if self.costs[bestindex] < self.best_solution.cost:
            best_schedule = schedules[bestindex]
            self.best_solution = Solution(
                route = self.routes[bestindex].numpy(),
                schedule = np.array(best_schedule),
                cost = best_schedule[-1]
            )
            self.max = self.Q * self.n / best_schedule[-1]
        
#             print("New best solution found!")
#             print(f"Best route: {self.best_solution.route}")
#             print(f"Best schedule: {self.best_solution.schedule}")
#             print(f"Best cost: {self.best_solution.cost}")
#         else:
#             print("No new best solution found.")
#         print('==========================')
    
    @torch.no_grad()
    def update_pheromone(self):
        self.pheromone = self.pheromone * self.decay

        best_route = self.best_solution.route
        self.pheromone[best_route[:-1], best_route[1:]] += self.Q / self.best_solution.cost

        if self.elitist:
            bestindex = self.costs.argmin()
            route = self.routes[bestindex]
            cost = self.costs[bestindex]
            self.pheromone[route[:-1], route[1:]] += self.Q / cost
        else:
            for route, cost in zip(self.routes, self.costs):
                self.pheromone[route[:-1], route[1:]] += self.Q / cost
        
        if self.min_max:
            self.pheromone[self.pheromone > self.max] = self.max
            self.pheromone[self.pheromone < self.min] = self.min

In [87]:
start_time = timeit.default_timer()
instance = read_RCPfile("X1_1.RCP")
schedule = SSGS(instance, list(range(len(instance))))
# print(schedule)
aco = ACO_RCPSP(instance, alpha=1.0, beta=2.0, gamma=1, elitist=True, min_max=True)
result = aco.run(810)

print()
print('schedule', result.schedule)
print()
print('route', result.route)
print()
print('pheromone.max', aco.pheromone.max())
print()
print('aco.max', aco.max)
print()
print('best_solution.cost', aco.best_solution.cost)
print('best_solution.schedule', aco.best_solution.schedule)
print('best_solution.route', aco.best_solution.route)
    
end_time = timeit.default_timer()
run_time = end_time - start_time

print("finished in seconds: " + str(run_time))


2 175
3 173
4 168
5 168
6 168
7 168
8 165
9 165
10 165
11 165
12 165
13 165
14 160
15 160
16 160
17 160
18 160
19 160
20 160
21 160
22 160
23 160
24 160
25 160
26 160
27 160
28 160
29 160
30 160
31 160
32 160
33 160
34 160
35 160
36 160
37 160
38 160
39 160
40 160
41 160
42 160
43 160
44 160
45 160
46 160
47 160
48 160
49 160
50 160
51 160
52 160
53 160
54 160
55 160
56 160
57 160
58 160
59 160
60 160
61 160
62 160
63 160
64 160
65 160
66 160
67 160
68 160
69 160
70 160
71 160
72 160
73 160
74 160
75 160
76 160
77 160
78 160
79 157
80 157
81 157
82 157
83 157
84 157
85 157
86 157
87 157
88 157
89 157
90 157
91 157
92 157
93 157
94 157
95 157
96 157
97 157
98 157
99 157
100 157
101 157
102 157
103 157
104 157
105 157
106 157
107 155
108 155
109 155
110 155
111 155
112 155
113 155
114 155
115 155
116 155
117 155
118 155
119 155
120 155
121 155
122 155
123 155
124 155
125 155
126 155
127 155
128 155
129 155
130 155
131 155
132 155
133 155
134 155
135 155
136 155
137 155
138 155
139 155
14